# Récupération des données du S&P500 et sauvegarde dans un CSV

Ce notebook récupère la liste des tickers du S&P500 depuis Wikipedia, appelle l'API Yahoo Finance pour récupérer un maximum d'informations sur chaque stock, y compris les données historiques pour une période spécifiée, et sauvegarde le tout dans un fichier CSV.

Pour indiquer la période souhaitée, modifiez les variables `start_date` et `end_date` dans la cellule suivante.

In [1]:
import os
import csv
import json
import yfinance as yf
import time
from datetime import datetime, timedelta

In [2]:
def recuperer_tickers_sp500():
    import pandas as pd
    url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    tables = pd.read_html(url, header=0)
    df = tables[0]
    tickers = df['Symbol'].tolist()
    return tickers



In [ ]:
def convert_date(value):
    """Convertit une valeur de date :
       - Si c'est un timestamp Unix (en secondes ou millisecondes), retourne 'YYYY-MM-DD'.
       - Si la valeur est vide, 'N/A' ou 'null', renvoie None.
       - Sinon, retourne la valeur telle quelle.
    """
    if value is None:
        return None
    if not isinstance(value, str):
        value = str(value)
    if not value or value.strip() in ["", "N/A", "null"]:
        return None
    if value.isdigit():
        try:
            # Si le timestamp a plus de 10 chiffres, on suppose qu'il est en millisecondes
            if len(value) > 10:
                ts = int(value) / 1000
            else:
                ts = int(value)
            return datetime.fromtimestamp(ts).strftime("%Y-%m-%d")
        except Exception as e:
            print(f"Erreur lors de la conversion du timestamp {value}: {e}")
            return None
    return value

def merge_historical(existing_list, new_list):
    """Fusionne deux listes d'enregistrements historiques en fonction du champ 'Date'.
       En cas de doublon, la donnée provenant de new_list écrase celle existante.
    """
    merged = {}
    for rec in existing_list:
        date = rec.get("Date")
        if date:
            merged[date] = rec
    for rec in new_list:
        date = rec.get("Date")
        if date:
            merged[date] = rec
    merged_list = list(merged.values())
    merged_list.sort(key=lambda r: r.get("Date"))
    return merged_list

def get_info_with_retry(ticker_obj, ticker, retries=2, delay=2):
    attempts = 0
    while attempts < retries:
        try:
            info = ticker_obj.info
            return info
        except Exception as e:
            print(f"Erreur lors de la récupération des informations pour {ticker}: {e}. Réessai dans {delay} secondes...")
            time.sleep(delay)
            attempts += 1
    print(f"⚠️ Échec de récupération des informations pour {ticker} après {retries} tentatives.")
    return {}

def get_history_with_retry(ticker_obj, ticker, start_date, end_date, retries=5, delay=5):
    attempts = 0
    while attempts < retries:
        try:
            history_df = ticker_obj.history(start=start_date, end=end_date, interval="1d")
            return history_df
        except Exception as e:
            print(f"Erreur lors de la récupération de l'historique pour {ticker}: {e}. Réessai dans {delay} secondes...")
            time.sleep(delay)
            attempts += 1
    print(f"⚠️ Échec de récupération de l'historique pour {ticker} après {retries} tentatives.")
    return None

def main():
    # Période pour les données historiques
    start_date_default = "2020-01-01"
    end_date = "2025-04-03"
    
    # Crée le dossier "stocks" s'il n'existe pas
    os.makedirs("stocks", exist_ok=True)

    # Récupère la date d'aujourd'hui au format AAAAMMJJ (ex: 20250319)
    today = datetime.today().strftime('%Y%m%d')

    # Construit le nom de fichier en y ajoutant la date
    csv_filename = os.path.join("stocks", f"sp500_stocks_info_{today}.csv")
    
    # Charger les données existantes (si elles existent) et indexer par ticker
    existing_data = {}
    if os.path.exists(csv_filename):
        with open(csv_filename, newline='', encoding='utf-8') as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                ticker = row.get("Ticker")
                if ticker:
                    existing_data[ticker] = row
    
    tickers = recuperer_tickers_sp500()
    updated_data = {}
    
    for ticker in tickers:
        print(f"Traitement de {ticker}...")
        stock_obj = yf.Ticker(ticker)
        # Récupérer les infos générales
        info = get_info_with_retry(stock_obj, ticker, retries=5, delay=5)
        
        # Déterminer la période de téléchargement des historiques
        if ticker in existing_data:
            existing_row = existing_data[ticker]
            historical_end_existing = existing_row.get("historical_end", "N/A")
            if historical_end_existing != "N/A" and historical_end_existing >= end_date:
                print(f"Les données historiques pour {ticker} sont déjà à jour.")
                updated_data[ticker] = existing_row
                continue
            else:
                if historical_end_existing != "N/A":
                    try:
                        last_date = datetime.strptime(historical_end_existing, "%Y-%m-%d")
                        new_start_date = (last_date + timedelta(days=1)).strftime("%Y-%m-%d")
                    except Exception as e:
                        print(f"Erreur lors de la conversion de historical_end pour {ticker}: {e}")
                        new_start_date = start_date_default
                else:
                    new_start_date = start_date_default
        else:
            new_start_date = start_date_default
        
        # Télécharger les données historiques manquantes
        history_df = get_history_with_retry(stock_obj, ticker, new_start_date, end_date, retries=5, delay=5)
        if history_df is not None:
            history_df.reset_index(inplace=True)
            new_history_json = history_df.to_json(orient="records")
            if not history_df.empty:
                new_history_start = history_df["Date"].min().strftime("%Y-%m-%d")
                new_history_end = history_df["Date"].max().strftime("%Y-%m-%d")
            else:
                new_history_start = "N/A"
                new_history_end = "N/A"
            try:
                new_history_list = json.loads(new_history_json)
            except Exception as e:
                print(f"Erreur lors du parsing JSON pour {ticker}: {e}")
                new_history_list = []
        else:
            new_history_list = []
            new_history_start = "N/A"
            new_history_end = "N/A"
        
        # Si le ticker existe déjà, fusionner les historiques
        if ticker in existing_data:
            existing_row = existing_data[ticker]
            historical_existing = existing_row.get("historical", "N/A")
            if historical_existing != "N/A":
                try:
                    existing_history_list = json.loads(historical_existing)
                except Exception as e:
                    print(f"Erreur lors du parsing de l'historique existant pour {ticker}: {e}")
                    existing_history_list = []
            else:
                existing_history_list = []
            merged_history = merge_historical(existing_history_list, new_history_list)
            if merged_history:
                merged_history_start = merged_history[0].get("Date", "N/A")
                merged_history_end = merged_history[-1].get("Date", "N/A")
            else:
                merged_history_start = "N/A"
                merged_history_end = "N/A"
            info["historical"] = json.dumps(merged_history)
            info["historical_start"] = merged_history_start
            info["historical_end"] = merged_history_end
        else:
            info["historical"] = json.dumps(new_history_list)
            info["historical_start"] = new_history_start
            info["historical_end"] = new_history_end
        
        info["Ticker"] = ticker
        updated_data[ticker] = info
        
        print(f"✅ Données traitées pour {ticker}")
        time.sleep(1)
    
    # Fusionner les données mises à jour avec celles qui n'ont pas été modifiées
    final_data = {**existing_data, **updated_data}
    all_stock_info = list(final_data.values())
    
    all_keys = set()
    for info in all_stock_info:
        all_keys.update(info.keys())
    all_keys = sorted(list(all_keys))
    
    with open(csv_filename, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=all_keys)
        writer.writeheader()
        for info in all_stock_info:
            writer.writerow(info)
    
    print(f"✅ Fichier CSV '{csv_filename}' généré avec succès.")

if __name__ == '__main__':
    main()


Traitement de MMM...
✅ Données traitées pour MMM
Traitement de AOS...
✅ Données traitées pour AOS
Traitement de ABT...
✅ Données traitées pour ABT
Traitement de ABBV...
✅ Données traitées pour ABBV
Traitement de ACN...
✅ Données traitées pour ACN
Traitement de ADBE...
✅ Données traitées pour ADBE
Traitement de AMD...
✅ Données traitées pour AMD
Traitement de AES...
✅ Données traitées pour AES
Traitement de AFL...
✅ Données traitées pour AFL
Traitement de A...
✅ Données traitées pour A
Traitement de APD...
✅ Données traitées pour APD
Traitement de ABNB...
✅ Données traitées pour ABNB
Traitement de AKAM...
✅ Données traitées pour AKAM
Traitement de ALB...
✅ Données traitées pour ALB
Traitement de ARE...
✅ Données traitées pour ARE
Traitement de ALGN...
✅ Données traitées pour ALGN
Traitement de ALLE...
✅ Données traitées pour ALLE
Traitement de LNT...
✅ Données traitées pour LNT
Traitement de ALL...
✅ Données traitées pour ALL
Traitement de GOOGL...
✅ Données traitées pour GOOGL
Traiteme

$BRK.B: possibly delisted; no timezone found


✅ Données traitées pour BRK.B
Traitement de BBY...
✅ Données traitées pour BBY
Traitement de TECH...
✅ Données traitées pour TECH
Traitement de BIIB...
✅ Données traitées pour BIIB
Traitement de BLK...
✅ Données traitées pour BLK
Traitement de BX...
✅ Données traitées pour BX
Traitement de BK...
✅ Données traitées pour BK
Traitement de BA...
✅ Données traitées pour BA
Traitement de BKNG...
✅ Données traitées pour BKNG
Traitement de BSX...
✅ Données traitées pour BSX
Traitement de BMY...
✅ Données traitées pour BMY
Traitement de AVGO...
✅ Données traitées pour AVGO
Traitement de BR...
✅ Données traitées pour BR
Traitement de BRO...
✅ Données traitées pour BRO
Traitement de BF.B...


$BF.B: possibly delisted; no price data found  (1d 2020-01-01 -> 2025-04-03)


✅ Données traitées pour BF.B
Traitement de BLDR...
✅ Données traitées pour BLDR
Traitement de BG...
✅ Données traitées pour BG
Traitement de BXP...
✅ Données traitées pour BXP
Traitement de CHRW...
✅ Données traitées pour CHRW
Traitement de CDNS...
✅ Données traitées pour CDNS
Traitement de CZR...
✅ Données traitées pour CZR
Traitement de CPT...
✅ Données traitées pour CPT
Traitement de CPB...
✅ Données traitées pour CPB
Traitement de COF...
✅ Données traitées pour COF
Traitement de CAH...
✅ Données traitées pour CAH
Traitement de KMX...
✅ Données traitées pour KMX
